In [212]:
import pandas as pd
import numpy as np
import math
import os
from datetime import datetime, date
from scipy import stats

In [22]:
cwd = os.getcwd()
path_split = cwd.split('/')

root_path = r'/'.join(path_split[:-1])

In [23]:
mi_path = root_path+'/extraction/mi_ipl.csv'
bb_path = root_path+'/extraction/bb_ipl.csv'

In [24]:
mi = pd.read_csv(mi_path)
bb = pd.read_csv(bb_path)

In [25]:
# Need to know which season each game was played in
season_lookup = mi[['match_id','season']]

In [285]:
data = bb.merge(season_lookup, how='inner', on='match_id')

In [287]:
data['four'] = np.where(data['runs'] == 4, True, False)
data['six'] = np.where(data['runs'] == 6, True, False)
data['boundary'] = data[['four','six']].any(1)

# Identifying overs bowled in the Powerplay
data['pp'] = np.where(data['over'] < 7, True, False)

In [289]:
# Aggregate by bowler, season and in or out of powerplay
grouped = data.groupby(['bowler','season','pp'], as_index=False).agg({'ball':'count',
                                                'runs':'sum',
                                                'extras':'sum',
                                                'bowling_wicket':'sum',
                                                'four':'sum',
                                                'six':'sum',
                                                'boundary':'sum'})

# Should we remove leg byes / byes from bowlers' stats?? Or just consider all round runs being given away
grouped['runs'] = grouped['runs']+grouped['extras']
grouped.drop(columns=['extras'], inplace=True)
grouped.rename(columns={'ball':'balls','bowling_wicket':'wickets'}, inplace=True)

In [290]:
# Calculating the rate at which runs are conceded and wickets are taken
for col in ['runs','wickets']:
    grouped[col+'_rate'] = grouped[col] / grouped['balls']

In [384]:
# Calculate the min and max for every combination of season / in or out in powerplay
minmax = grouped.groupby(['season','pp']).agg({'runs_rate':['min','max'], 'wickets_rate':['min','max']}).T.to_dict()

In [385]:
def get_scores(row, minmax):
    season = row['season']
    pp = row['pp']

    for col in ['runs_rate','wickets_rate']:
        value = row[col]
        minv = minmax[season,pp][col,'min']
        maxv = minmax[season,pp][col,'max']

        if col == 'runs_rate':
            # Run rate is 1 - because it's better to have a low score
            row[col+'_scaled'] = 1 - ((value-minv) / (maxv-minv))
        else:
            # scale wickets rate, obviously higher wicket rate == good
            row[col+'_scaled'] = ((value-minv) / (maxv-minv))

    return row

In [417]:
# get scaled scores for run rate and wicket rate
scores = grouped.apply(lambda row: get_scores(row, minmax), axis=1)

In [418]:
# find a list of all the seasons we have data for
seasons = scores.sort_values('season').season.unique().tolist()

In [419]:
# creates a fibonacci sequence to be used as a recency multipler (ignores the 0 as didn't want to multiply scores by 0)
def fibonacci(n):
    n0 = 0
    n1 = 1
    seq = [n1]

    while len(seq) < n:
        newn = n0+n1
        n0 = n1
        n1 = newn
        seq.append(newn)

    maxv = max(seq)

    scaled_seq = [i/maxv for i in seq]

    return scaled_seq

In [420]:
# join year:recency multiplier
recency = dict(zip(seasons, fibonacci(len(seasons))))

In [421]:
# get this into a column
scores['recency'] = scores['season'].apply(lambda x: recency[x])

In [422]:
# create a multiplier column, if the runs/wickets are in the powerplay, multiply by 1.2
scores['pp_multiplier'] = np.where(scores['pp'], 1.2, 1)

In [423]:
# Runs score and wickets score
# Sum together for players' powerplay or non-powerplay season score

scores['runs_score'] = scores['pp_multiplier'] * scores['runs_rate_scaled'] * scores['recency']
scores['wickets_score'] = scores['pp_multiplier'] * scores['wickets_rate_scaled'] * scores['recency'] #* 1.1
scores['score'] = scores['runs_score'] + scores['wickets_score']

In [444]:
# Sum a players scores across seasons and in/out powerplay
rankings = scores.groupby('bowler').agg({'score':'sum','season':['min','max']})

# Player must have appeared in the IPL 2019 or 2020 -- removes old players who have retired
rankings = rankings[rankings[('season','max')].isin([2019,2020])].sort_values(('score','sum'), ascending=False)

In [445]:
# Give rank in a column
rankings['rank'] = rankings['score'].rank(ascending=False)

In [447]:
rankings.head(10)

score season        rank
                     sum    min   max      
bowler                                     
JJ Bumrah       6.334356   2013  2020   1.0
Rashid Khan     6.138528   2017  2020   2.0
YS Chahal       6.121210   2013  2020   3.0
R Tewatia       5.852520   2014  2020   4.0
PP Chawla       5.796256   2008  2020   5.0
A Mishra        5.755992   2008  2020   6.0
B Kumar         5.689965   2011  2020   7.0
Sandeep Sharma  5.616222   2013  2020   8.0
TA Boult        5.613694   2015  2020   9.0
CH Morris       5.550952   2013  2020  10.0